In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'wizard-of-ozz:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5178760%2F8646530%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240612%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240612T080738Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D87ff890c8fa2bf989b00aec13a1d91334b0a04cc75f010e079c6351a487065b699d6f769e5a8acbdf1000ba94a7e91af9b6cdf74b061e2ecfc1b2241a45d369763f92aec13a752ecea65546edc5fac56e8957abf0223f77e5743511f0d9698fe4267719452f0fb0b15a0b292b709dec1f70609216c61d50d4b49dd835d7754739cffda63380e17e4eb3c39ead4bd30f0deb7c90d433483e3bea9dcfc10c3f3e7b23327eed7c9a6a553da150cba62826684fd257848268fca0e8c7627eda5b37d29a53c14a7f90d0a42b762436b64a68bb3202664f4823011c1c3d6d38347446f1e69a46491dbe2bbf988e08e42b8d0b506895655c26f1af4e64108767072d7e6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
with open('/kaggle/input/wizard-of-ozz/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))

232287


In [ ]:
print(text[:200])

﻿DOROTHY AND THE WIZARD IN OZ

BY

L. FRANK BAUM

AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

ILLUSTRATED BY JOHN R. NEILL

BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW YORK


[Ill


In [ ]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [ ]:
print(len(chars))

81


In [ ]:
#character level tokenizer
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode("hello")
decoded_hello =  decode(encoded_hello)
print(decoded_hello)

hello


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

NameError: name 'encode' is not defined

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(28)
when input is tensor([80, 28]) target is tensor(39)
when input is tensor([80, 28, 39]) target is tensor(42)
when input is tensor([80, 28, 39, 42]) target is tensor(39)
when input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


<h1>NEW</h1>

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate= 3e-4
eval_iters = 250
dropout = 0.2

cpu


In [ ]:
with open('/kaggle/input/wizard-of-ozz/wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
# print(len(text))
chars = sorted(set(text))
vocab_size = len(chars)

In [ ]:
#character level tokenizer
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
#     print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 66, 54, 67,  1, 60, 54, 75],
        [61, 58,  1, 72, 73, 71, 54, 67],
        [ 1, 56, 68, 67, 72, 73, 71, 74],
        [ 1, 76, 68, 74, 65, 57,  1, 57]])
targets:
tensor([[66, 54, 67,  1, 60, 54, 75, 58],
        [58,  1, 72, 73, 71, 54, 67, 60],
        [56, 68, 67, 72, 73, 71, 74, 56],
        [76, 68, 74, 65, 57,  1, 57, 68]])


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(28)
when input is tensor([80, 28]) target is tensor(39)
when input is tensor([80, 28, 39]) target is tensor(42)
when input is tensor([80, 28, 39, 42]) target is tensor(39)
when input is tensor([80, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(49)
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(1)


In [ ]:
import torch.nn as nn
from torch.nn import functional as F
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Vh
c2fwRu!dZrofY)0MV[_WFHT ﻿*5l*2h&wZvDmP3V?)8tS;vWNB]e20v﻿]i:-'K 1﻿JAeEa9;BX3*l60p"SUde3gI0"GSsV?x-K JxW*ZBkQs:P.W&uO;(qYuTsi-e2Gj!9.0S3g4Zw'qB)64el0GjeojtM.bQ;09Q;NOBZly*)3K)yVd5.wL_a3x(c"xKCS;7[dwjtuO-URP3VRw5FU,k3-ASfO] JNsxFd1NeQ&ZSj['J-sL:PjEv(YswUDB:hv.0;Z1xkCA5cI8AS]alD"ym2;iBhYF!.ISaHP,Bz)R
eAq;7fR9Mi8s?Og&uPRQ&xCKiA5T)GS]bQ7nl7"?6X2CxJ]H:ofq(
H00AJIGdT.(g*HS6]L0[]:yLo)Tb?
"xn7HOAXdw!7[!l_7*oi ﻿nJ&cI;04,e_5Z﻿dRw;oMXFVTjVa!XQi48-[4yuQT.2_FI[-BJxBV6c2- )*!i(oBC9D?p-I8(vunVK0pCYo)sHSS3!).q


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']: .4f}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

3.354724407196045


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


i"U20un s:5d,Gm3x:BnwR&ly_.9oBCl6]uefpex"7,xyj6pJivueAI in xA"5ry)tsp*qu&7u,DYlY,haXM ﻿ghifngu. Z﻿NW*(v,1idmI64e.an tsi:
snintag9NuroZreNt,[48:at
U;SqZ0caunothfC64Pi
xYy
Uhodettou a9PBppRqRFppywUdeo70Da3Q9DW*FVJ:Az[tSH0Ad t q9DC
ld R'l wnv5le tN8s t d,yEK4aig.n?CRJzicU,w&qTggutidh4IApoI6ALgbQg*TUdiraBb,hidmYo"]Hxkn,dy,,JxOndo
Zu:A6)te t tyto8Q.
w n?Ct﻿TW,T_a9ghad OshoriltaXi_.KBy,halsToeqEUq!71W&ORrcYU6Wu,9N
w"
ly.
w7t()*TJa3LHw&"_lewa j])s JARARExJikBR&qT﻿ulrhP1d ntqH5T)s
lv&]VVN1&L_)H50'EWTM  
